In [ ]:
# gbm_discrete_survival.py
# Requirements: pandas, numpy, scikit-learn, lightgbm
# pip install pandas numpy scikit-learn lightgbm

import pandas as pd
import numpy as np
from math import ceil
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import log_loss
# Optional: concordance index from lifelines (for ranking)
from lifelines.utils import concordance_index

# ------------------------------
# 1) Expand function (discrete intervals)
# ------------------------------
def expand_to_intervals(df, interval_days=7, max_days=None, cap_intervals=None):
    """
    Expand listing-level dataset to discrete intervals for pooled-logistic (GBM) survival.
    Input df must contain: listing_id, listing_date, days_to_lease, event (0/1), plus static features.
    Returns expanded_df with one row per (listing_id, interval_index).
    """
    required = {"listing_id", "listing_date", "days_to_lease", "event"}
    if not required.issubset(set(df.columns)):
        raise ValueError(f"Missing required columns: {required - set(df.columns)}")
    if max_days is None:
        max_days = int(df["days_to_lease"].max())
    max_intervals = ceil(max_days / interval_days) if max_days>0 else 1
    if cap_intervals is not None:
        max_intervals = min(max_intervals, cap_intervals)
    rows = []
    feature_cols = [c for c in df.columns if c not in ["listing_date", "days_to_lease", "event"]]
    for _, r in df.iterrows():
        n_intervals = int(ceil(max(1, r["days_to_lease"]) / interval_days))
        n_intervals = min(n_intervals, max_intervals)
        for t in range(1, n_intervals+1):
            start_day = (t-1)*interval_days + 1
            end_day = t*interval_days
            # event occurs in final interval only for events
            event_interval = 1 if (int(r["event"]) == 1 and t == n_intervals) else 0
            rec = {
                "listing_id": r["listing_id"],
                "listing_date": r["listing_date"],
                "interval_index": t,
                "interval_start_day": start_day,
                "interval_end_day": end_day,
                "time_elapsed_days": min(end_day, int(r["days_to_lease"]))
            }
            for c in feature_cols:
                rec[c] = r[c]
            rec["event_interval"] = event_interval
            rows.append(rec)
    expanded = pd.DataFrame(rows)
    # reorder
    front = ["listing_id","listing_date","interval_index","interval_start_day","interval_end_day","time_elapsed_days","event_interval"]
    others = [c for c in expanded.columns if c not in front]
    return expanded[front+others]

# ------------------------------
# 2) Example: prepare train/val split (time-based)
# ------------------------------
# Assume `df` exists: listing-level dataframe for 2018-2024 with matured labels
# Example: df = pd.read_parquet("listing_ttl.parquet")

# 1) Choose a cutoff date for train/validation split (by listing_date)
#    e.g., train on listings before '2023-07-01', val on listings in [2023-07-01, 2024-06-30]
train_cutoff = pd.to_datetime("2023-07-01")

train_listings = df[df["listing_date"] < train_cutoff].copy()
val_listings = df[df["listing_date"] >= train_cutoff].copy()

# 2) Optionally cap max_days to limit extreme tail expansion (keeps training small)
interval_days = 7
max_days = 180  # expand up to 180 days (tunable)
cap_intervals = ceil(max_days / interval_days)

# 3) Expand
train_expanded = expand_to_intervals(train_listings, interval_days=interval_days, max_days=max_days, cap_intervals=cap_intervals)
val_expanded = expand_to_intervals(val_listings, interval_days=interval_days, max_days=max_days, cap_intervals=cap_intervals)

# ------------------------------
# 3) Features & label for GBM
# ------------------------------
# Choose features (static as-of-listing_date) + time feature(s)
static_features = ["relative_price", "views_7d", "inquiries_7d", "maint_90", "bedrooms", "sqft"]
# Add the interval index/time elapsed as features to allow time-varying baseline hazard
time_features = ["interval_index", "time_elapsed_days"]
feature_cols = static_features + time_features

# Optionally create interactions or engineered time features:
# e.g., interval_index and interval_index^2, log-time, etc.
train_expanded["interval_index_sq"] = train_expanded["interval_index"]**2
val_expanded["interval_index_sq"] = val_expanded["interval_index"]**2
feature_cols += ["interval_index_sq"]

# ------------------------------
# 4) Handle imbalance / weighting
# ------------------------------
# There are many more negative rows (no event) than positives.
# One approach: weight positive examples higher, or use is_unbalance in LGBM.
# Another: sample negatives for speed but keep positives.
pos_weight = (train_expanded["event_interval"]==0).sum() / max(1, (train_expanded["event_interval"]==1).sum())
# You can pass 'is_unbalance' or 'scale_pos_weight' to LightGBM; we'll use is_unbalance param.
# Or create a weight column:
train_expanded["sample_weight"] = 1.0
val_expanded["sample_weight"] = 1.0
# Increase weight for positives (optional)
train_expanded.loc[train_expanded["event_interval"]==1, "sample_weight"] = pos_weight

# ------------------------------
# 5) Train LightGBM per-interval classifier
# ------------------------------
lgb_params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "learning_rate": 0.05,
    "num_leaves": 64,
    "min_data_in_leaf": 50,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbosity": -1,
    "seed": 42,
    # "is_unbalance": True,  # alternative to manual weights
}

dtrain = lgb.Dataset(train_expanded[feature_cols], label=train_expanded["event_interval"], weight=train_expanded["sample_weight"])
dval = lgb.Dataset(val_expanded[feature_cols], label=val_expanded["event_interval"], reference=dtrain)

bst = lgb.train(lgb_params, dtrain, num_boost_round=1000, valid_sets=[dtrain, dval], early_stopping_rounds=50)

# Save model
bst.save_model("gbm_discrete_survival.txt")

# ------------------------------
# 6) Predict hazards and compute survival for each listing
# ------------------------------
# Predict per-row hazard (probability of event in that interval)
val_expanded["pred_hazard"] = bst.predict(val_expanded[feature_cols], num_iteration=bst.best_iteration)

# Compute survival by listing: S(t) = prod_{k<=t} (1 - h_k)
val_expanded = val_expanded.sort_values(["listing_id", "interval_index"])
val_expanded["one_minus_h"] = 1.0 - val_expanded["pred_hazard"]
val_expanded["survival"] = val_expanded.groupby("listing_id")["one_minus_h"].cumprod()

# Probability leased within t intervals = 1 - survival(t)
val_expanded["p_leased_within_interval"] = 1.0 - val_expanded["survival"]

# Example: extract P(leased within 30 days) depending on interval length
horizon_days = 30
horizon_interval = ceil(horizon_days / interval_days)
p30 = val_expanded[val_expanded["interval_index"] == horizon_interval][["listing_id","p_leased_within_interval"]].set_index("listing_id")
# p30 is the per-listing probability leased within ~30 days

# ------------------------------
# 7) Evaluation ideas
# ------------------------------
# - Evaluate per-interval logloss on validation set:
val_logloss = log_loss(val_expanded["event_interval"], val_expanded["pred_hazard"])
print("Validation per-interval logloss:", val_logloss)

# - For ranking (who leases sooner), compute concordance index using predicted risk (1 - survival at some time)
#   Here we compute predicted median risk up to 90 days as a single score for ordering:
pred_score = val_expanded[val_expanded["interval_index"] <= ceil(90/interval_days)].groupby("listing_id")["p_leased_within_interval"].max()
# Build arrays for concordance: ground truth is days_to_lease and event from original validation listings
val_meta = val_listings.set_index("listing_id")
# Align
common = pred_score.index.intersection(val_meta.index)
c_index = concordance_index(val_meta.loc[common,"days_to_lease"], -pred_score.loc[common].values, val_meta.loc[common,"event"])
print("Concordance index (higher better):", c_index)

# - Brier score / calibration: compute predicted survival vs observed at time horizons (requires grouping by listing).
# See scikit-survival or custom implementations.

# ------------------------------
# 8) Notes & production tips
# ------------------------------
# - Use interval length thoughtfully: 7 days (weekly) is common; shorter intervals => bigger dataset.
# - Use time features to let model learn baseline hazard (interval_index, time_elapsed_days, seasonality flags).
# - For time-varying covariates (e.g., views_7d), you can recompute feature as-of each interval and include it per interval row.
# - Keep splitting by listing_date to avoid leakage.
# - Use LightGBM's "is_unbalance" or sample weights for class imbalance.
# - When scoring current active listings, expand them into intervals out to desired horizon (e.g., 90 days),
#   predict hazards, and compute survival curves.
